todo: 
- letní filmy, co už nedávají (víno)
- záhada hlavolamu je seriál

In [4]:
import os
import polars as pl

In [5]:
pl.Config(tbl_rows=200)

In [6]:
from calendar import monthrange

In [7]:
def monthday(date_value):
    # Here date_value is a datetime.datetime object, not a Polars column
    month_length = monthrange(date_value.year, date_value.month)[1]
    return date_value.month + (date_value.day / month_length)

In [8]:
df = pl.read_parquet(
    os.path.join("data","fdb.parquet")
).with_columns(
    pl.col("datum").dt.month().alias("mesic")
).with_columns(
    pl.col("datum").dt.week().alias("tyden")
).with_columns(
    pl.col("datum").dt.year().alias("rok")
).with_columns(
    pl.when(
        pl.col("url").str.contains("serial/")
    ).then(
        pl.lit("serial")
    ).when(
        pl.col("url").str.contains("film/")
    ).then(
        pl.lit("film")
    ).alias(
        "kategorie"
    )
).sort(
    by="datum"
)

sys:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance


In [9]:
df.head(2)

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""ČT2""",2007-01-01 10:05:00,"""Dotek motýla (?) [TV film]""","""05.10.2020 v 12:30 (ČT 3)""","""Další díl/opakování:05.10.2020…","""film/dotek-motyla/39629""",1,1,2007,"""film"""
"""ČT1""",2007-01-01 18:05:00,"""O velkém nosu (?) [TV inscenac…","""01.06.2024 v 13:05 (ČT1)""","""Další díl/opakování:01.06.2024…","""film/o-velkem-nosu/41025""",1,1,2007,"""film"""


In [10]:
df.tail(2)

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""Prima Show""",2025-04-17 23:55:00,"""Tyhle šaty beru!""","""19.04.2025 v 06:00 (Prima Show…","""Další díl/opakování:19.04.2025…","""film/tyhle-saty-beru-say-yes-t…",4,16,2025,"""film"""
"""Nova Fun""",2025-04-17 23:55:00,"""Dva a půl chlapa""","""18.04.2025 v 12:15 (Nova Fun)1…","""Další díl/opakování:18.04.2025…","""serial/dva-a-pul-chlapa-two-an…",4,16,2025,"""serial"""


In [11]:
df.sample(10).select(pl.col("noticka")).to_series().to_list()

['Další díl/opakování:12.12.2022 v 08:30 (ČT 3)',
 'Další díl/opakování:17.10.2024 v 08:35 (Prima LOVE)18.10.2024 v 08:35 (Prima LOVE)19.10.2024 v 09:05 (Prima LOVE)20.10.2024 v 07:40 (Prima LOVE)21.10.2024 v 08:45 (Prima LOVE)22.10.2024 v 08:40 (Prima LOVE)23.10.2024 v 08:45 (Prima LOVE)24.10.2024 v 08:45 (Prima LOVE)',
 'Další díl/opakování:16.04.2025 v 20:10 (Prima STAR)17.04.2025 v 04:15 (Prima)17.04.2025 v 04:15 (Prima MAX)17.04.2025 v 20:10 (Prima STAR)18.04.2025 v 04:30 (Prima)18.04.2025 v 04:45 (Prima MAX)18.04.2025 v 20:10 (Prima STAR)19.04.2025 v 03:50 (Prima)',
 'Další díl/opakování:12.04.2025 v 10:40 (Prima LOVE)13.04.2025 v 10:55 (Prima LOVE)13.04.2025 v 11:50 (Prima)14.04.2025 v 23:55 (Prima)19.04.2025 v 10:40 (Prima LOVE)20.04.2025 v 10:50 (Prima LOVE)20.04.2025 v 12:00 (Prima)',
 'Další díl/opakování:16.04.2025 v 11:45 (NOVA)16.04.2025 v 13:00 (Nova Lady)17.04.2025 v 11:45 (NOVA)17.04.2025 v 13:05 (Nova Lady)18.04.2025 v 13:05 (Nova Lady)22.04.2025 v 11:50 (NOVA)22.04.2

In [12]:
kanaly = df.select(pl.col("kanal")).unique().to_series().to_list()
kanaly

['Prima',
 'Nova Action',
 'Cinemax',
 'Nova Lady',
 'Prima Show',
 'Film Europe Channel',
 'ČT2',
 'CS Film',
 'ČT1',
 'Cinemax2',
 'Film+',
 'Prima LOVE',
 'Nova 2',
 'NOVA',
 'Prima STAR',
 'Nova Cinema',
 'TV Barrandov',
 'Prima Cool',
 'ČT 3',
 'Nova Fun',
 'VOYO',
 'CNN Prima News']

In [13]:
verejnopravni = [k for k in kanaly if 'ČT' in k]
print(verejnopravni)
soukrome = [k for k in kanaly if 'ČT' not in k]
print(soukrome)

['ČT2', 'ČT1', 'ČT 3']
['Prima', 'Nova Action', 'Cinemax', 'Nova Lady', 'Prima Show', 'Film Europe Channel', 'CS Film', 'Cinemax2', 'Film+', 'Prima LOVE', 'Nova 2', 'NOVA', 'Prima STAR', 'Nova Cinema', 'TV Barrandov', 'Prima Cool', 'Nova Fun', 'VOYO', 'CNN Prima News']


In [14]:
vysilane_verejnopravni = df.filter(pl.col("kanal").is_in(verejnopravni)).select(pl.col("nazev")).unique().to_series().to_list()
vysilane_soukromymi = df.filter(pl.col("kanal").is_in(soukrome)).select(pl.col("nazev")).unique().to_series().to_list()
vysilane_vsude = [x for x in vysilane_verejnopravni if x in vysilane_soukromymi]

In [15]:
aspon_na_dvou = df.group_by("nazev").agg(pl.col("kanal").unique().len()).filter(pl.col("kanal") >= 2).select(pl.col("nazev")).to_series().to_list()

In [16]:
v_poslednich_dvou_letech = df.filter(pl.col("datum").dt.year().is_in([2023,2024])).filter(pl.col("datum").dt.month().is_in([7,8])).select(pl.col("nazev")).unique().to_series().to_list()

In [17]:
x_let = df.with_columns(pl.col('datum').dt.year().alias('rok')).group_by('nazev').agg(pl.col('rok').unique().len()).filter(pl.col('rok') >= 5).select(pl.col('nazev')).unique().to_series().to_list()

In [18]:
vsude_a_x_let_pracovni = [x for x in x_let if x in aspon_na_dvou]
vsude_a_x_let = [x for x in vsude_a_x_let_pracovni if x in v_poslednich_dvou_letech]

In [19]:
zajima_nas = ['ČT1', 'ČT2', 'NOVA', 'Prima']

In [20]:
letni = df.filter(pl.col("rok").is_between(2015,2024)).filter(pl.col('kanal').is_in(zajima_nas)).filter(pl.col('nazev').is_in(vsude_a_x_let) & pl.col('mesic').is_in([7,8])).group_by('nazev').len().rename({'len':'v_lete'})

In [21]:
zimni = df.filter(pl.col("rok").is_between(2015,2024)).filter(pl.col('kanal').is_in(zajima_nas)).filter(pl.col('nazev').is_in(vsude_a_x_let) & (~pl.col('mesic').is_in([7,8]))).group_by('nazev').len().rename({'len':'v_zime'})

In [22]:
zimni = df.filter(pl.col("rok").is_between(2015,2024)).filter(pl.col('kanal').is_in(zajima_nas)).filter(pl.col('nazev').is_in(vsude_a_x_let) & (~pl.col('mesic').is_in([6,7,8,9]))).group_by('nazev').len().rename({'len':'v_zime'})

In [23]:
kategorie = df.filter(pl.col("rok").is_between(2015,2024)).filter(pl.col("nazev").is_in(vsude_a_x_let)).group_by("nazev").agg(pl.col("kategorie").mode()).explode("kategorie").drop_nulls()
kategorie = df.filter(pl.col("rok").is_between(2015,2024)).filter(pl.col("nazev").is_in(vsude_a_x_let)).group_by(['nazev','kategorie']).len().sort(by='len',descending=True).unique(subset=['nazev'], keep='first').drop('len').drop_nulls()
kategorie

nazev,kategorie
str,str
"""Hartova válka""","""film"""
"""Návrat""","""film"""
"""Rambo: Poslední krev""","""film"""
"""Po čem ženy touží""","""film"""
"""Jedenácté přikázání""","""film"""
"""Láska a jiné závislosti""","""film"""
"""Televarieté""","""film"""
"""MasterChef Česko""","""film"""
"""U Pěti veverek""","""film"""


In [24]:
pomer = letni.join(zimni, how='full', on='nazev').with_columns(
    pl.when(
        pl.col('nazev').is_null()
    ).then(
        pl.col('nazev_right')
    ).otherwise(
        pl.col('nazev')
    ).alias(
        'nazev'
    )
).with_columns(
    pl.col('v_lete').fill_null(0)
).drop(
    'nazev_right'
).with_columns(
    (pl.col('v_lete') / pl.col('v_zime')).alias('pomer')
)

In [25]:
pomer.filter(pl.col("nazev").str.contains("Lepší už"))

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Lepší už to nebude""",5,null,null


In [26]:
nechceme = [
    "Šaráda",
    "Záchranáři",
    "Máma",
    "Helena",
    "Hodinový manžel",
    "Uvolněte se, prosím",
    "Vinaři",
    "Kameňák",
    "Etiketa",
    "Za oponou"
]

In [27]:
filmy = kategorie.filter(pl.col('kategorie') == 'film').filter(~pl.col('nazev').is_in(nechceme)).select(pl.col('nazev')).to_series().to_list()
serialy = kategorie.filter(pl.col('kategorie') == 'serial').filter(~pl.col('nazev').is_in(nechceme)).select(pl.col('nazev')).to_series().to_list()

In [28]:
pomer.filter(pl.col('nazev').str.contains('Hříšný t'))

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Hříšný tanec""",4,7,0.571429


In [29]:
pomer.filter(pl.col('nazev').str.contains('Pomád'))

nazev,v_lete,v_zime,pomer
str,u32,u32,f64


In [30]:
pomer.filter(pl.col('nazev').str.contains('Vinneto'))

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Vinnetou - Rudý gentleman""",10,2,5.0
"""Vinnetou a Old Firehand""",6,1,6.0
"""Vinnetou - Poslední výstřel""",10,2,5.0
"""Vinnetou a míšenka Apanači""",12,2,6.0
"""Vinnetou""",16,2,8.0


In [31]:
pomer.filter(pl.col('nazev').str.contains('Shatter'))

nazev,v_lete,v_zime,pomer
str,u32,u32,f64


In [32]:
letni_filmy = pomer.filter(pl.col('nazev').is_in(filmy)).filter(pl.col('v_lete') >= 8).sort(by='pomer',descending=True).with_columns(pl.lit("film").alias("kategorie")).head(50)
letni_filmy

nazev,v_lete,v_zime,pomer,kategorie
str,u32,u32,f64,str
"""Indiana Jones a Chrám zkázy""",13,null,null,"""film"""
"""Indiana Jones a Poslední křížo…",13,null,null,"""film"""
"""Indiana Jones a Království kři…",13,null,null,"""film"""
"""2Bobule""",14,null,null,"""film"""
"""Indiana Jones a dobyvatelé ztr…",13,null,null,"""film"""
"""Bobule""",14,null,null,"""film"""
"""Skryté zločiny osmašedesátého""",10,null,null,"""film"""
"""Petrolejový princ""",12,1,12.0,"""film"""
"""Zralé víno""",11,1,11.0,"""film"""


In [33]:
letni_serialy = pomer.filter(pl.col('nazev').is_in(serialy)).filter(pl.col('v_lete') >= 25).sort(by='pomer',descending=True).with_columns(pl.lit("seriál").alias("kategorie")).head(50)
letni_serialy

nazev,v_lete,v_zime,pomer,kategorie
str,u32,u32,f64,str
"""Malý pitaval z velkého města""",177,3,59.0,"""seriál"""
"""Chalupáři""",123,20,6.15,"""seriál"""
"""Dobrá Voda""",56,10,5.6,"""seriál"""
"""Pan Tau""",61,13,4.692308,"""seriál"""
"""Správná dvojka""",40,15,2.666667,"""seriál"""
"""Spongebob v kalhotách""",42,17,2.470588,"""seriál"""
"""Návštěvníci""",26,11,2.363636,"""seriál"""
"""Můj přítel Monk""",616,303,2.033003,"""seriál"""
"""Nemocnice na kraji města""",104,52,2.0,"""seriál"""


In [34]:
letni_televize = pl.concat(
    [letni_filmy, letni_serialy]
).sort(by='pomer',descending=True).filter((pl.col('pomer') >= 2.2) | (pl.col('pomer').is_null()))

In [35]:
letni_televize

nazev,v_lete,v_zime,pomer,kategorie
str,u32,u32,f64,str
"""Indiana Jones a Chrám zkázy""",13,null,null,"""film"""
"""Indiana Jones a Poslední křížo…",13,null,null,"""film"""
"""Indiana Jones a Království kři…",13,null,null,"""film"""
"""2Bobule""",14,null,null,"""film"""
"""Indiana Jones a dobyvatelé ztr…",13,null,null,"""film"""
"""Bobule""",14,null,null,"""film"""
"""Skryté zločiny osmašedesátého""",10,null,null,"""film"""
"""Malý pitaval z velkého města""",177,3,59.0,"""seriál"""
"""Petrolejový princ""",12,1,12.0,"""film"""


In [36]:
pomer.filter(~pl.col('nazev').is_in(serialy) & ~pl.col('nazev').is_in(filmy)).filter(pl.col('v_lete') >= 50).sort(by='pomer',descending=True).head(20)

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Máma""",85,12,7.083333
"""Etiketa""",598,164,3.646341
"""Hodinový manžel""",51,15,3.4
"""Vinaři""",150,49,3.061224
"""Kameňák""",63,21,3.0
"""Svět zvířat""",52,88,0.590909
"""Žiješ jenom 2x""",251,504,0.498016
"""V zajetí železné opony""",115,359,0.320334
"""Ta naša kapela""",96,316,0.303797


In [37]:
nechceme_urls = ['serial/kamenak/163648']

In [38]:
vyber_strojovy = letni_televize.filter(~pl.col('nazev').is_in(nechceme)).select(pl.col('nazev')).to_series().to_list()

graf = df.filter(
    pl.col("rok").is_between(2015,2024)
).filter(
    pl.col('nazev').is_in(vyber_strojovy)
).filter(
    ~pl.col('url').is_in(nechceme_urls)
).filter(
    pl.col("kanal").is_in(zajima_nas)
).group_by(
    ['nazev','tyden']
).len().join(
    kategorie, on="nazev", how="left"
).with_columns(
    pl.when(
        (pl.col('nazev') == 'Skryté zločiny osmašedesátého') | (pl.col('nazev') == 'Jsou hvězdy, které nehasnou')
    ).then(pl.lit("serial")).otherwise(pl.col("kategorie")).alias("kategorie")
).with_columns(
    pl.col('kategorie').replace_strict(
        {'film':'filmy','serial':'seriály a pořady'}
    )
)

graf.sort(by=['nazev','tyden'])

nazev,tyden,len,kategorie
str,i8,u32,str
"""2Bobule""",29,1,"""filmy"""
"""2Bobule""",30,1,"""filmy"""
"""2Bobule""",32,2,"""filmy"""
"""2Bobule""",33,10,"""filmy"""
"""2Bobule""",37,3,"""filmy"""
"""Angelika a král""",29,1,"""filmy"""
"""Angelika a král""",31,2,"""filmy"""
"""Angelika a král""",32,3,"""filmy"""
"""Angelika a král""",33,2,"""filmy"""


In [39]:
graf.filter(pl.col("nazev").str.contains("Malý pitaval"))

nazev,tyden,len,kategorie
str,i8,u32,str
"""Malý pitaval z velkého města""",28,21,"""seriály a pořady"""
"""Malý pitaval z velkého města""",39,2,"""seriály a pořady"""
"""Malý pitaval z velkého města""",34,15,"""seriály a pořady"""
"""Malý pitaval z velkého města""",26,12,"""seriály a pořady"""
"""Malý pitaval z velkého města""",32,22,"""seriály a pořady"""
"""Malý pitaval z velkého města""",40,2,"""seriály a pořady"""
"""Malý pitaval z velkého města""",29,23,"""seriály a pořady"""
"""Malý pitaval z velkého města""",33,21,"""seriály a pořady"""
"""Malý pitaval z velkého města""",30,24,"""seriály a pořady"""


In [40]:
len(vyber_strojovy)

49

In [41]:
import altair as alt

In [42]:
df.sample(10)

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""CS Film""",2024-11-06 07:00:00,"""O modrém autíčku""","""09.01.2025 v 07:00 (CS Film)10…","""Další díl/opakování:09.01.2025…","""serial/o-modrem-auticku/102125""",11,45,2024,"""serial"""
"""CS Film""",2023-08-21 12:00:00,"""Plavecký mariáš""","""03.08.2024 v 17:05 (Prima)04.0…","""Další díl/opakování:03.08.2024…","""film/plavecky-marias/15925""",8,34,2023,"""film"""
"""Prima""",2017-07-16 04:15:00,"""Utta Danella: Duhový sen""",null,"""Romantický film N (2007). Hani…","""serial/utta-danella-12-dil-utt…",7,28,2017,"""serial"""
"""VOYO""",2017-05-01 07:05:00,"""Krtkova dobrodružství""","""27.03.2024 v 07:20 (ČT :D / ČT…","""Další díl/opakování:27.03.2024…","""film/krtkova-dobrodruzstvi/155…",5,18,2017,"""film"""
"""Nova Action""",2023-03-13 22:15:00,"""Policie Chicago""","""21.04.2025 v 13:40 (Nova Gold)…","""Další díl/opakování:21.04.2025…","""serial/policie-chicago-chicago…",3,11,2023,"""serial"""
"""Prima""",2008-02-20 17:45:00,"""Minuty regionu""",null,null,"""film/minuty-regionu/83029""",2,8,2008,"""film"""
"""ČT2""",2016-09-27 19:50:00,"""Zprávy v českém znakovém jazyc…",null,null,null,9,39,2016,null
"""Film+""",2014-11-21 05:05:00,"""Upoutávky z vysílání Film+""",null,null,null,11,47,2014,null
"""Nova Cinema""",2023-01-27 10:05:00,"""Kriminálka Miami""","""09.09.2024 v 13:35 (JOJ)10.09.…","""Další díl/opakování:09.09.2024…","""serial/kriminalka-miami-csi-mi…",1,4,2023,"""serial"""


In [43]:
from datetime import datetime

In [44]:
roky = range(2007,2025)
prazdniny = []
for r in roky:
    prazdniny.append({'zacatek':datetime(year=r,month=7,day=1), 'konec':datetime(year=r,month=8,day=31)})

prazdniny = pl.DataFrame(prazdniny).with_columns(pl.col('zacatek').dt.week()).with_columns(pl.col('konec').dt.week())
prazdniny

df_prazdniny = []
for x in range(1,54):
    df_prazdniny.append({'tyden':x, 'prazdniny':len(prazdniny.filter((pl.col('zacatek') <= x) & (pl.col('konec') >= x)))})

df_prazdniny = pl.DataFrame(df_prazdniny).with_columns(pl.col('prazdniny') / len(roky)).filter(pl.col("prazdniny") != 0).with_columns(
    pl.col("tyden").min().alias("zacatek")
).with_columns(
    pl.col("tyden").max().alias("konec")
)
df_prazdniny

tyden,prazdniny,zacatek,konec
i64,f64,i64,i64
26,0.555556,26,36
27,1.0,26,36
28,1.0,26,36
29,1.0,26,36
30,1.0,26,36
31,1.0,26,36
32,1.0,26,36
33,1.0,26,36
34,1.0,26,36


In [45]:
prazdniny.filter((pl.col('zacatek') <= 26) & (pl.col('konec') >= 26))

zacatek,konec
i8,i8
26,35
26,35
26,35
26,35
26,35
26,35
26,35
26,35
26,35


In [46]:
df_prazdniny

tyden,prazdniny,zacatek,konec
i64,f64,i64,i64
26,0.555556,26,36
27,1.0,26,36
28,1.0,26,36
29,1.0,26,36
30,1.0,26,36
31,1.0,26,36
32,1.0,26,36
33,1.0,26,36
34,1.0,26,36


In [47]:
terminy = alt.Chart(
    df_prazdniny
).mark_rect(color='#D5B574', opacity=0.1
).encode(
    alt.X('zacatek:Q'),
    alt.X2('konec:Q'),
)

terminy

alt.Chart(...)

In [48]:
df.filter(pl.col("nazev").str.contains("Skryté zločiny osma"))

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""ČT1""",2008-08-19 21:15:00,"""Skryté zločiny osmašedesátého""",null,null,null,8,34,2008,null
"""ČT1""",2008-08-27 02:15:00,"""Skryté zločiny osmašedesátého""",null,null,null,8,35,2008,null
"""ČT2""",2009-08-18 21:20:00,"""Skryté zločiny osmašedesátého""",null,null,null,8,34,2009,null
"""ČT2""",2009-08-22 08:30:00,"""Skryté zločiny osmašedesátého""",null,null,null,8,34,2009,null
"""ČT2""",2010-08-19 17:25:00,"""Skryté zločiny osmašedesátého""",null,"""Stovky zraněných, desítky mrtv…",null,8,33,2010,null
"""ČT2""",2013-08-22 03:35:00,"""Skryté zločiny osmašedesátého""",null,"""Dokument (2008). Stovky zraněn…",null,8,34,2013,null
"""ČT2""",2014-08-19 20:55:00,"""Skryté zločiny osmašedesátého""",null,"""Dokument (2008). Opakování 27.…",null,8,34,2014,null
"""ČT2""",2014-08-28 01:10:00,"""Skryté zločiny osmašedesátého""",null,"""Dokument (2008). Stovky zraněn…",null,8,35,2014,null
"""ČT2""",2016-08-22 21:20:00,"""Skryté zločiny osmašedesátého""",null,"""Dokument (2008). Opakování 24.…",null,8,34,2016,null


In [49]:
df.filter(pl.col("nazev").str.contains("Zralé víno"))

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""CS Film""",2008-04-28 18:00:00,"""Zralé víno (?)""","""20.01.2025 v 20:00 (CS Film)""","""Další díl/opakování:20.01.2025…","""film/zrale-vino/24097""",4,18,2008,"""film"""
"""CS Film""",2008-05-30 22:00:00,"""Zralé víno (?)""","""20.01.2025 v 20:00 (CS Film)""","""Další díl/opakování:20.01.2025…","""film/zrale-vino/24097""",5,22,2008,"""film"""
"""TV Barrandov""",2009-11-20 20:05:00,"""Zralé víno""","""20.01.2025 v 20:00 (CS Film)""","""Další díl/opakování:20.01.2025…","""film/zrale-vino/24097""",11,47,2009,"""film"""
"""TV Barrandov""",2009-11-21 09:15:00,"""Zralé víno""","""20.01.2025 v 20:00 (CS Film)""","""Další díl/opakování:20.01.2025…","""film/zrale-vino/24097""",11,47,2009,"""film"""
"""CS Film""",2010-01-22 20:00:00,"""Zralé víno (?)""","""20.01.2025 v 20:00 (CS Film)""","""Další díl/opakování:20.01.2025…","""film/zrale-vino/24097""",1,3,2010,"""film"""
"""TV Barrandov""",2010-03-14 15:15:00,"""Zralé víno""","""20.01.2025 v 20:00 (CS Film)""","""Další díl/opakování:20.01.2025…","""film/zrale-vino/24097""",3,10,2010,"""film"""
"""CS Film""",2010-04-23 20:00:00,"""Zralé víno""","""20.01.2025 v 20:00 (CS Film)""","""Další díl/opakování:20.01.2025…","""film/zrale-vino/24097""",4,16,2010,"""film"""
"""CS Film""",2010-09-03 20:00:00,"""Zralé víno""","""20.01.2025 v 20:00 (CS Film)""","""Další díl/opakování:20.01.2025…","""film/zrale-vino/24097""",9,35,2010,"""film"""
"""CS Film""",2011-04-15 14:00:00,"""Zralé víno""","""20.01.2025 v 20:00 (CS Film)""","""Další díl/opakování:20.01.2025…","""film/zrale-vino/24097""",4,15,2011,"""film"""


In [50]:
import warnings
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [51]:
obsah = alt.Chart(
    graf,
    title=alt.Title(
        'Co o prázdninách vysílá ČT1, ČT2, Nova a Prima',
        subtitle=[
            'Filmy, seriály a pořady, které v minulosti vysílaly alespoň dvě stanice, běžely v televizi',
            'v letech 2023 a 2024, a které jsou posledních deset let v červenci a srpnu vysílanější ',
            'alespoň 2,2× častěji než od října do května.'
        ]
    )
).mark_circle(
).encode(
    alt.X('tyden:Q', axis=alt.Axis(title="týden v roce", tickColor='#F1D1BB', tickOpacity=0.6, tickWidth = 0.5), scale=alt.Scale(
        domainMin = 1,
        domainMax = 52,
        padding=0,        # Remove extra padding
        nice=False        # Prevent extending to "nice" numbers
    )),
    alt.Y('nazev:N', sort=vyber_strojovy, axis=alt.Axis(title=None, tickColor='#F1D1BB', tickOpacity=0.6, tickWidth = 0.5)),
    alt.Size('len:Q', scale=alt.Scale(range=[25,300]), legend=None),
    alt.Color('kategorie:N', scale=alt.Scale(range=['#D15A41','#788A98']), legend=alt.Legend(title=None, orient="top", direction="horizontal"))
)

finale = alt.layer(terminy, obsah).configure_view(
    stroke='transparent'
).configure_axis(grid=True, gridOpacity=0.6, gridWidth = 0.5, gridColor='#F1D1BB').properties(
    width=230,
    height=850
)

finale

alt.LayerChart(...)

In [52]:
me_to_neurazi(graf=finale,kredity='vizualizace: iROZHLAS.cz | zdroj programů: fbd.cz', soubor='letni_porady')

<figure>
    <a href="https://data.irozhlas.cz/grafy/letni_porady.svg" target="_blank">
    <img src="https://data.irozhlas.cz/grafy/letni_porady.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [53]:
df.filter(pl.col("nazev") == "Za oponou")

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""ČT1""",2007-04-05 20:00:00,"""Za oponou""",null,null,null,4,14,2007,null
"""ČT1""",2007-04-08 03:45:00,"""Za oponou""",null,null,null,4,14,2007,null
"""ČT1""",2007-04-09 02:20:00,"""Za oponou""",null,null,null,4,15,2007,null
"""ČT1""",2007-04-26 20:00:00,"""Za oponou""",null,null,null,4,17,2007,null
"""ČT1""",2007-04-29 03:10:00,"""Za oponou""",null,null,null,4,17,2007,null
"""ČT1""",2007-04-30 10:05:00,"""Za oponou""",null,null,null,4,18,2007,null
"""ČT1""",2007-06-16 20:00:00,"""Za oponou""",null,null,null,6,24,2007,null
"""ČT2""",2007-06-17 23:35:00,"""Za oponou""",null,null,null,6,24,2007,null
"""ČT1""",2007-10-03 20:00:00,"""Za oponou""",null,null,null,10,40,2007,null


## Pořady, které zmizely

In [55]:
nebyly_v_poslednich_dvou_letech = [x for x in vsude_a_x_let_pracovni if x not in v_poslednich_dvou_letech]

In [56]:
nebyly_v_poslednich_dvou_letech

['Italské prázdniny',
 'Perfektní manžel',
 'Pohádkář',
 'Dámská jízda',
 'Star Wars: Epizoda V - Impérium vrací úder',
 'Princ a chuďas',
 'Moonraker',
 'Láska a válka',
 'Černý úsvit',
 'Z Jihlavy do hlavy',
 'Slavnosti sněženek',
 'Astro Boy',
 'Pěsti ze železa',
 'Žít a být',
 'Rande v Římě',
 'Povídka malostranská',
 'Výchova dívek v Čechách',
 'Utta Danella: Tajemství naší lásky',
 'Paralelní světy',
 'Král Škorpion',
 'Ohlédnutí',
 'Mezi dveřmi je A.G.',
 'Rambo',
 'Dixie jako Brno',
 'Doba ledová 2 - Obleva',
 'Podivné dědictví',
 'První republika',
 'Amazing Spider-Man 2',
 'Zlatí úhoři',
 'Elba stopami císaře',
 'Dostaveníčko na Vysočině',
 'Šmankote, babičko, čaruj!',
 'Vinobraní',
 'Po stopách amazonských',
 'Pohádka o dešťové víle',
 'Klikař Charlie',
 '7 životů',
 'Zlatíčko',
 'Balto',
 'Noc oživlých mrtvol',
 'Nic než pravda',
 'Stupnice strachu',
 'Úsměvy Evy Sadkové',
 'Vražda na Eigeru',
 'Dárky',
 'Krásy evropského pobřeží: Golfský proud',
 'Město nemrtvých',
 'Jíst,

In [57]:
letni_kdysi = df.filter(pl.col('kanal').is_in(zajima_nas)).filter(pl.col('nazev').is_in(nebyly_v_poslednich_dvou_letech) & pl.col('mesic').is_in([7,8])).group_by('nazev').len().rename({'len':'v_lete'})
zimni_kdysi = df.filter(pl.col('kanal').is_in(zajima_nas)).filter(pl.col('nazev').is_in(nebyly_v_poslednich_dvou_letech) & (~pl.col('mesic').is_in([7,8]))).group_by('nazev').len().rename({'len':'v_zime'})

In [58]:
pomer_nevysilane = letni_kdysi.join(zimni_kdysi, how='full', on='nazev').with_columns(
    pl.when(
        pl.col('nazev').is_null()
    ).then(
        pl.col('nazev_right')
    ).otherwise(
        pl.col('nazev')
    ).alias(
        'nazev'
    )
).with_columns(
    pl.col('v_lete').fill_null(0)
).drop(
    'nazev_right'
).with_columns(
    (pl.col('v_lete') / pl.col('v_zime')).alias('pomer')
)

In [59]:
pomer_nevysilane.filter(pl.col('pomer') > 1.8).sort(by=['pomer','v_lete'],descending=True)

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Bob a Bobek na cestách""",43,3,14.333333
"""Karel Gott v Lucerně""",13,1,13.0
"""Na cestě po Jindřichohradecku""",9,1,9.0
"""Fotbal""",9,1,9.0
"""Na cestě po Podhorácku""",8,1,8.0
"""Pohádky z lesa""",8,1,8.0
"""Na cestě po Beskydech""",7,1,7.0
"""Na cestě po Pardubicku""",7,1,7.0
"""Na cestě po Vitorazsku""",7,1,7.0


In [167]:
df.filter(pl.col("nazev").str.contains("Četník ze St. Tropez"))

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""NOVA""",2009-12-26 15:30:00,"""Četník ze St. Tropez""","""08.02.2025 v 22:45 (ČT1)""","""Další díl/opakování:08.02.2025…","""film/cetnik-ze-saint-tropez-le…",12,52,2009,"""film"""
"""NOVA""",2010-10-02 16:30:00,"""Četník ze St. Tropez""","""08.02.2025 v 22:45 (ČT1)""","""Další díl/opakování:08.02.2025…","""film/cetnik-ze-saint-tropez-le…",10,39,2010,"""film"""
"""NOVA""",2012-01-02 17:45:00,"""Četník ze St. Tropez""","""08.02.2025 v 22:45 (ČT1)""","""Další díl/opakování:08.02.2025…","""film/cetnik-ze-saint-tropez-le…",1,1,2012,"""film"""
"""Nova Cinema""",2012-01-03 14:25:00,"""Četník ze St. Tropez""","""13.01.2024 v 22:35 (ČT1)18.01.…","""Další díl/opakování:13.01.2024…","""film/cetnik-ze-saint-tropez-le…",1,1,2012,"""film"""
"""NOVA""",2013-01-02 17:40:00,"""Četník ze St. Tropez""","""08.02.2025 v 22:45 (ČT1)""","""Další díl/opakování:08.02.2025…","""film/cetnik-ze-saint-tropez-le…",1,1,2013,"""film"""
"""Nova Cinema""",2013-01-03 15:40:00,"""Četník ze St. Tropez""","""13.01.2024 v 22:35 (ČT1)18.01.…","""Další díl/opakování:13.01.2024…","""film/cetnik-ze-saint-tropez-le…",1,1,2013,"""film"""
"""VOYO""",2016-08-31 09:05:00,"""Četník ze St. Tropez""","""13.01.2024 v 22:35 (ČT1)18.01.…","""Další díl/opakování:13.01.2024…","""film/cetnik-ze-saint-tropez-le…",8,35,2016,"""film"""
"""VOYO""",2016-09-01 10:45:00,"""Četník ze St. Tropez""","""13.01.2024 v 22:35 (ČT1)18.01.…","""Další díl/opakování:13.01.2024…","""film/cetnik-ze-saint-tropez-le…",9,35,2016,"""film"""
"""VOYO""",2016-09-02 10:35:00,"""Četník ze St. Tropez""","""13.01.2024 v 22:35 (ČT1)18.01.…","""Další díl/opakování:13.01.2024…","""film/cetnik-ze-saint-tropez-le…",9,35,2016,"""film"""


In [157]:
df.filter(pl.col("nazev") == "Turner a Hooch")

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""Prima""",2009-12-21 11:35:00,"""Turner a Hooch""","""17.12.2023 v 10:35 (Film+)""","""Další díl/opakování:17.12.2023…","""film/turner-a-hooch-turner-hoo…",12,52,2009,"""film"""
"""Prima Cool""",2010-05-23 16:15:00,"""Turner a Hooch""","""17.12.2023 v 10:35 (Film+)""","""Další díl/opakování:17.12.2023…","""film/turner-a-hooch-turner-hoo…",5,20,2010,"""film"""
"""NOVA""",2015-08-08 11:35:00,"""Turner a Hooch""","""17.12.2023 v 10:35 (Film+)""","""Další díl/opakování:17.12.2023…","""film/turner-a-hooch-turner-hoo…",8,32,2015,"""film"""
"""NOVA""",2015-08-09 01:10:00,"""Turner a Hooch""","""17.12.2023 v 10:35 (Film+)""","""Další díl/opakování:17.12.2023…","""film/turner-a-hooch-turner-hoo…",8,32,2015,"""film"""
"""Nova Cinema""",2015-08-09 07:35:00,"""Turner a Hooch""","""17.12.2023 v 10:35 (Film+)""","""Další díl/opakování:17.12.2023…","""film/turner-a-hooch-turner-hoo…",8,32,2015,"""film"""
"""NOVA""",2016-07-02 08:40:00,"""Turner a Hooch""","""17.12.2023 v 10:35 (Film+)""","""Další díl/opakování:17.12.2023…","""film/turner-a-hooch-turner-hoo…",7,26,2016,"""film"""
"""NOVA""",2016-07-03 03:15:00,"""Turner a Hooch""","""17.12.2023 v 10:35 (Film+)""","""Další díl/opakování:17.12.2023…","""film/turner-a-hooch-turner-hoo…",7,26,2016,"""film"""
"""Nova Cinema""",2016-07-04 05:30:00,"""Turner a Hooch""","""17.12.2023 v 10:35 (Film+)""","""Další díl/opakování:17.12.2023…","""film/turner-a-hooch-turner-hoo…",7,27,2016,"""film"""
"""NOVA""",2017-07-29 09:35:00,"""Turner a Hooch""","""17.12.2023 v 10:35 (Film+)""","""Další díl/opakování:17.12.2023…","""film/turner-a-hooch-turner-hoo…",7,30,2017,"""film"""


In [60]:
df.filter(pl.col("nazev") == "Tajemství proutěného košíku")

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""ČT1""",2011-02-20 13:55:00,"""Tajemství proutěného košíku""","""13.02.2022 v 15:45 (ČT 3)14.02…","""Další díl/opakování:13.02.2022…","""serial/tajemstvi-prouteneho-ko…",2,7,2011,"""serial"""
"""ČT1""",2011-02-27 13:55:00,"""Tajemství proutěného košíku""","""13.02.2022 v 15:45 (ČT 3)14.02…","""Další díl/opakování:13.02.2022…","""serial/tajemstvi-prouteneho-ko…",2,8,2011,"""serial"""
"""ČT1""",2011-03-06 14:15:00,"""Tajemství proutěného košíku""","""13.02.2022 v 15:45 (ČT 3)14.02…","""Další díl/opakování:13.02.2022…","""serial/tajemstvi-prouteneho-ko…",3,9,2011,"""serial"""
"""ČT1""",2011-03-13 14:05:00,"""Tajemství proutěného košíku""","""13.02.2022 v 15:45 (ČT 3)14.02…","""Další díl/opakování:13.02.2022…","""serial/tajemstvi-prouteneho-ko…",3,10,2011,"""serial"""
"""ČT1""",2011-03-20 14:05:00,"""Tajemství proutěného košíku""","""13.02.2022 v 15:45 (ČT 3)14.02…","""Další díl/opakování:13.02.2022…","""serial/tajemstvi-prouteneho-ko…",3,11,2011,"""serial"""
"""ČT1""",2011-03-27 14:00:00,"""Tajemství proutěného košíku""","""13.02.2022 v 15:45 (ČT 3)14.02…","""Další díl/opakování:13.02.2022…","""serial/tajemstvi-prouteneho-ko…",3,12,2011,"""serial"""
"""ČT1""",2011-04-03 14:05:00,"""Tajemství proutěného košíku""","""13.02.2022 v 15:45 (ČT 3)14.02…","""Další díl/opakování:13.02.2022…","""serial/tajemstvi-prouteneho-ko…",4,13,2011,"""serial"""
"""ČT1""",2011-04-10 07:00:00,"""Tajemství proutěného košíku""","""13.02.2022 v 15:45 (ČT 3)14.02…","""Další díl/opakování:13.02.2022…","""serial/tajemstvi-prouteneho-ko…",4,14,2011,"""serial"""
"""ČT1""",2011-04-17 07:00:00,"""Tajemství proutěného košíku""","""13.02.2022 v 15:45 (ČT 3)14.02…","""Další díl/opakování:13.02.2022…","""serial/tajemstvi-prouteneho-ko…",4,15,2011,"""serial"""


In [61]:
pomer.filter(pl.col('nazev').str.contains('Hříšný ta'))

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Hříšný tanec""",4,7,0.571429


In [62]:
pomer.filter(pl.col('nazev').str.contains('z městečk'))

nazev,v_lete,v_zime,pomer
str,u32,u32,f64


In [63]:
df.filter(pl.col('nazev').str.contains('z městečk'))

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""ČT1""",2007-07-02 16:30:00,"""My holky z městečka (?) [TV mi…","""31.08.2024 v 14:25 (ČT :D / ČT…","""Další díl/opakování:31.08.2024…","""film/my-holky-z-mestecka/41510""",7,27,2007,"""film"""
"""ČT2""",2007-07-04 14:55:00,"""My holky z městečka (?) [TV mi…","""31.08.2024 v 14:25 (ČT :D / ČT…","""Další díl/opakování:31.08.2024…","""film/my-holky-z-mestecka/41510""",7,27,2007,"""film"""
"""ČT1""",2007-07-09 16:30:00,"""My holky z městečka (?) [TV mi…","""31.08.2024 v 14:25 (ČT :D / ČT…","""Další díl/opakování:31.08.2024…","""film/my-holky-z-mestecka/41510""",7,28,2007,"""film"""
"""ČT2""",2007-07-11 14:40:00,"""My holky z městečka (?) [TV mi…","""31.08.2024 v 14:25 (ČT :D / ČT…","""Další díl/opakování:31.08.2024…","""film/my-holky-z-mestecka/41510""",7,28,2007,"""film"""
"""ČT1""",2007-07-16 16:30:00,"""My holky z městečka (?) [TV mi…","""31.08.2024 v 14:25 (ČT :D / ČT…","""Další díl/opakování:31.08.2024…","""film/my-holky-z-mestecka/41510""",7,29,2007,"""film"""
"""ČT2""",2007-07-18 14:50:00,"""My holky z městečka (?) [TV mi…","""31.08.2024 v 14:25 (ČT :D / ČT…","""Další díl/opakování:31.08.2024…","""film/my-holky-z-mestecka/41510""",7,29,2007,"""film"""
"""ČT1""",2007-07-23 16:30:00,"""My holky z městečka (?) [TV mi…","""31.08.2024 v 14:25 (ČT :D / ČT…","""Další díl/opakování:31.08.2024…","""film/my-holky-z-mestecka/41510""",7,30,2007,"""film"""
"""ČT2""",2007-07-25 14:40:00,"""My holky z městečka (?) [TV mi…","""31.08.2024 v 14:25 (ČT :D / ČT…","""Další díl/opakování:31.08.2024…","""film/my-holky-z-mestecka/41510""",7,30,2007,"""film"""
"""ČT1""",2007-07-30 16:30:00,"""My holky z městečka (?) [TV mi…","""31.08.2024 v 14:25 (ČT :D / ČT…","""Další díl/opakování:31.08.2024…","""film/my-holky-z-mestecka/41510""",7,31,2007,"""film"""


## (dis)kontinuita

In [65]:
df.filter(pl.col('nazev').str.contains('Krotitel')).sample(10)

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""Cinemax""",2020-04-16 16:10:00,"""Krotitel Morant""","""20.07.2020 v 03:30 (Cinemax)""","""Další díl/opakování:20.07.2020…","""film/krotitel-morant-breaker-m…",4,16,2020,"""film"""
"""Cinemax""",2018-01-25 06:00:00,"""Krotitelé duchů""","""26.01.2025 v 07:45 (HBO3)""","""Další díl/opakování:26.01.2025…","""film/krotitele-duchu-ii-ghostb…",1,4,2018,"""film"""
"""ČT2""",2010-11-13 01:40:00,"""Krotitelé dluhů""","""III (9/16)""","""Po smrti manželky zjistil, že …",null,11,45,2010,null
"""ČT1""",2009-12-15 21:40:00,"""Krotitelé dluhů""",null,null,null,12,51,2009,null
"""Nova Cinema""",2014-09-15 06:55:00,"""Krotitelé duchů 2""","""26.01.2025 v 07:45 (HBO3)""","""Další díl/opakování:26.01.2025…","""film/krotitele-duchu-ii-ghostb…",9,38,2014,"""film"""
"""ČT1""",2012-05-02 22:30:00,"""Krotitelé dluhů""",null,null,"""film/krotitele-dluhu/84924""",5,18,2012,"""film"""
"""NOVA""",2010-01-17 09:35:00,"""Krotitelé duchů 2""","""09.04.2025 v 12:05 (HBO2)""","""Další díl/opakování:09.04.2025…","""film/krotitele-duchu-ii-ghostb…",1,2,2010,"""film"""
"""Cinemax""",2018-01-24 15:10:00,"""Krotitelé duchů""","""26.01.2025 v 07:45 (HBO3)""","""Další díl/opakování:26.01.2025…","""film/krotitele-duchu-ii-ghostb…",1,4,2018,"""film"""
"""Cinemax2""",2020-02-05 04:05:00,"""Krotitel Morant""","""20.07.2020 v 03:30 (Cinemax)""","""Další díl/opakování:20.07.2020…","""film/krotitel-morant-breaker-m…",2,6,2020,"""film"""


In [66]:
df.select(pl.col('nazev')).sample(20)

nazev
str
"""Simpsonovi"""
"""Pravdivé lži"""
"""Bláznivé lásky"""
"""Cyklotoulky"""
"""Čas probuzení"""
"""Áčko"""
"""Těžká váha"""
"""Dva a půl chlapa"""
"""Anděl na horách"""


In [67]:
def calculate_ratio(current_row: pl.Series, previous_row: pl.Series) -> float:
    current_items = set(current_row)
    previous_items = set(previous_row)

    if not previous_items:
        return 0.0

    common_items = current_items.intersection(previous_items)
    ratio = len(common_items) / len(previous_items)
    return ratio


In [68]:
vyvoj_ct1 = df.filter(pl.col('kanal') == 'ČT1').group_by(['rok','tyden']).agg(pl.col("nazev")).sort(by=['rok','tyden'])

In [69]:
ratios = []
for i in range(1, len(vyvoj_ct1)):
    current_row = vyvoj_ct1["nazev"][i]
    previous_row = vyvoj_ct1["nazev"][i - 1]
    ratio = calculate_ratio(current_row, previous_row)
    ratios.append(ratio)

In [70]:
ratios.insert(0, float('nan'))

In [71]:
vyvoj_ct1 = vyvoj_ct1.with_columns(pl.Series("ratio", ratios))

In [72]:
vyvoj_ct1.filter(pl.col('rok').is_between(2008,2024)).group_by('tyden').agg(pl.col('ratio').median())

tyden,ratio
i8,f64
3,0.666667
21,0.688889
49,0.677083
41,0.68
31,0.663158
2,0.491803
15,0.653465
36,0.564815
44,0.671875


In [73]:
alt.Chart(
    vyvoj_ct1.filter(pl.col('rok').is_between(2008,2024)).group_by('tyden').agg(pl.col('ratio').median()), width=800
).mark_bar(
).encode(
    alt.X('tyden:Q'),
    alt.Y('ratio:Q')
)

alt.Chart(...)

In [74]:
def calculate_ratio(current_row: pl.Series, previous_row: pl.Series) -> float:
    current_items = set(current_row)
    previous_items = set(previous_row)

    if not previous_items:
        return 0.0

    common_items = current_items.intersection(previous_items)
    ratio = len(common_items) / len(previous_items)
    return ratio
